In [ ]:
!pip install sagemaker

In [ ]:
import requests

In [ ]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_deserializer
from fastai.vision import *
import time

In [ ]:
import tarfile
with tarfile.open('models/model_v1.tar.gz', 'w:gz') as f:
    t = tarfile.TarInfo('models')
    t.type = tarfile.DIRTYPE
    f.addfile(t)
    f.add('models/export.pkl', arcname='export.pkl')

In [ ]:
import sagemaker
from sagemaker.utils import name_from_base
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
role

In [ ]:
bucket

In [ ]:
bucket

In [ ]:
prefix = f'sagemaker/{name_from_base("model_v1")}'
model_artefact = sagemaker_session.upload_data(path=str('models/model_v1.tar.gz'), bucket=bucket, key_prefix=prefix)

RealTimePredictor class to accept jpeg images as input and output JSON. The default behaviour is to accept a numpy array.

In [ ]:
class ImagePredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=None, 
                         deserializer=json_deserializer, content_type='image/jpeg')

In [ ]:
role = sagemaker.get_execution_role()

In [ ]:
model=PyTorchModel(model_data=model_artefact, name=name_from_base("model_v1"),
    role=role, framework_version='1.3.1', source_dir='./',entry_point='serve.py', predictor_cls=ImagePredictor)

In [ ]:
then=time.time()
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
now=time.time()

In [ ]:
print(f'{(now-then)/60} mins')

In [ ]:
url = "https://wpdlaunchpaddeploy.s3.amazonaws.com/3842R___IMG_4755.jpg"
img_bytes = requests.get(url).content

In [ ]:
then=time.time()
print(predictor.predict(img_bytes))
now=time.time()

## If data is in JSON
Just define the content type as some string that you are going to verify in the input_fn. For example here we defined it as `application/json` which is defined as `JSON_CONTENT_TYPE` in the `serve.py` script and hence the neccesary logic to handle this type of data can be implemented in the input_fn()

In [ ]:
import json

In [ ]:
data={'url':url,'data':"someinfo"}

In [ ]:
predictor.content_type='application/json'

In [ ]:
then=time.time()
print(predictor.predict(json.dumps(data)))
now=time.time()

In [ ]:
print(now-then)

## To delete the endpoint and model in code

In [ ]:
predictor.delete_endpoint()
predictor.delete_model()